In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler
import tensorflow as tf
from pathlib import Path
from time import strftime

In [2]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs
print('Tensorboard launched at localhost:6006')

Launching TensorBoard...

Tensorboard launched at localhost:6006


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
def get_run_dir(root='my_logs'):
    return Path(root) / strftime('run_%Y_%m_%d_%H_%M_%S')

In [5]:
def get_score(true_values, predicted_values):
    mae_ = mean_absolute_error(true_values, predicted_values)
    mse_ = mean_squared_error(true_values, predicted_values)
    r2_ = r2_score(true_values, predicted_values)

    return mae_, mse_, r2_

# Loading data

In [6]:
dataset = pd.read_csv('dataset.csv')

In [7]:
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :-3]
labels = dataset.iloc[:, -2:]

In [8]:
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2, random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full, test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

# Scaling personality values to range 0 - 1

In [9]:
mms = MinMaxScaler()

data_train_mms = mms.fit_transform(data_train)
data_train_mms = pd.DataFrame(data_train_mms, columns=data_train.columns)
data_validation_mms = mms.fit_transform(data_validation)
data_validation_mms = pd.DataFrame(data_validation_mms, columns=data_validation.columns)

# Scaling input values to Gaussian distributions

In [10]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train_mms)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train_mms.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation_mms)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation_mms.columns)

# Training

In [11]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
hidden1 = tf.keras.layers.Dense(64, activation='relu')(normalized)
hidden2 = tf.keras.layers.Dense(128, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(64, activation='relu')(hidden2)
output = tf.keras.layers.Dense(2)(hidden3)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 normalization (Normalizatio  (None, 13)               27        
 n)                                                              
                                                                 
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                             

2022-11-20 18:39:41.067693: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-20 18:39:41.745404: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 34 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1


In [12]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              metrics=['mae'])

In [13]:
norm_layer.adapt(data_train_qt)

In [14]:
n_epochs = 100

run_dir = get_run_dir()
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir, profile_batch=(0, n_epochs))

history = model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation), callbacks=[tensorboard_cb])

Epoch 1/100
570/570 [==============================] - 4s 5ms/step - loss: 6.8004 - mae: 2.1205 - val_loss: 6.2682 - val_mae: 2.0844
Epoch 2/100
570/570 [==============================] - 3s 5ms/step - loss: 4.3401 - mae: 1.7537 - val_loss: 6.0776 - val_mae: 2.0622
Epoch 3/100
570/570 [==============================] - 3s 5ms/step - loss: 4.1981 - mae: 1.7211 - val_loss: 5.9782 - val_mae: 2.0477
Epoch 4/100
570/570 [==============================] - 3s 4ms/step - loss: 4.1468 - mae: 1.7071 - val_loss: 6.0228 - val_mae: 2.0579
Epoch 5/100
570/570 [==============================] - 4s 6ms/step - loss: 4.1210 - mae: 1.7010 - val_loss: 6.0031 - val_mae: 2.0572
Epoch 6/100
570/570 [==============================] - 3s 5ms/step - loss: 4.1059 - mae: 1.6974 - val_loss: 5.9698 - val_mae: 2.0565
Epoch 7/100
570/570 [==============================] - 2s 4ms/step - loss: 4.0947 - mae: 1.6951 - val_loss: 5.9074 - val_mae: 2.0398
Epoch 8/100
570/570 [==============================] - 3s 4ms/step - 

In [15]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

215/215 [==============================] - 1s 3ms/step

Values for validation set:
MAE: 1.9736607412070768
MSE: 5.66496265263298
RMSE: 2.3801182013994557
R2:  0.012279776079626936


# Overfitting?

In [16]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
output = tf.keras.layers.Dense(2)(normalized)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 13)]              0         
                                                                 
 normalization_1 (Normalizat  (None, 13)               27        
 ion)                                                            
                                                                 
 dense_4 (Dense)             (None, 2)                 28        
                                                                 
Total params: 55
Trainable params: 28
Non-trainable params: 27
_________________________________________________________________


In [17]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])

In [18]:
norm_layer.adapt(data_train_qt)

In [19]:
n_epochs = 50

# run_dir = get_run_dir()
# tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir, profile_batch=(0, n_epochs))

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/50
570/570 [==============================] - 3s 5ms/step - loss: 30.0115 - mae: 5.0177 - val_loss: 19.0710 - val_mae: 3.7066
Epoch 2/50
570/570 [==============================] - 3s 6ms/step - loss: 24.5716 - mae: 4.4759 - val_loss: 18.5070 - val_mae: 3.6420
Epoch 3/50
570/570 [==============================] - 3s 5ms/step - loss: 20.1233 - mae: 3.9710 - val_loss: 17.4588 - val_mae: 3.5369
Epoch 4/50
570/570 [==============================] - 3s 4ms/step - loss: 16.4271 - mae: 3.5198 - val_loss: 14.8005 - val_mae: 3.2372
Epoch 5/50
570/570 [==============================] - 2s 4ms/step - loss: 13.3648 - mae: 3.1331 - val_loss: 12.3481 - val_mae: 2.9564
Epoch 6/50
570/570 [==============================] - 2s 4ms/step - loss: 10.8617 - mae: 2.8060 - val_loss: 10.6179 - val_mae: 2.7437
Epoch 7/50
570/570 [==============================] - 3s 5ms/step - loss: 8.8575 - mae: 2.5333 - val_loss: 8.7973 - val_mae: 2.5116
Epoch 8/50
570/570 [==============================] - 4s 6ms/ste

In [20]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

215/215 [==============================] - 0s 2ms/step

Values for validation set:
MAE: 1.9943835264988259
MSE: 5.711783566684479
RMSE: 2.3899337996447683
R2:  -0.0016500872448512727


# Check if data from all subjects works better

In [6]:
dataset = pd.read_csv('dataset_full.csv')
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :-3]
labels = dataset.iloc[:, -2:]
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2,
                                                                              random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full,
                                                                                test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

In [7]:
mms = MinMaxScaler()

data_train_mms = mms.fit_transform(data_train)
data_train_mms = pd.DataFrame(data_train_mms, columns=data_train.columns)
data_validation_mms = mms.fit_transform(data_validation)
data_validation_mms = pd.DataFrame(data_validation_mms, columns=data_validation.columns)

In [8]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train_mms)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train_mms.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation_mms)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation_mms.columns)

In [24]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
output = tf.keras.layers.Dense(2)(normalized)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 13)]              0         
                                                                 
 normalization_2 (Normalizat  (None, 13)               27        
 ion)                                                            
                                                                 
 dense_5 (Dense)             (None, 2)                 28        
                                                                 
Total params: 55
Trainable params: 28
Non-trainable params: 27
_________________________________________________________________


In [25]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])

In [26]:
norm_layer.adapt(data_train_qt)

In [27]:
n_epochs = 50

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/50
2394/2394 [==============================] - 7s 3ms/step - loss: 22.3434 - mae: 4.1966 - val_loss: 14.1423 - val_mae: 3.1827
Epoch 2/50
2394/2394 [==============================] - 7s 3ms/step - loss: 9.5151 - mae: 2.6135 - val_loss: 7.2848 - val_mae: 2.3094
Epoch 3/50
2394/2394 [==============================] - 6s 2ms/step - loss: 4.9067 - mae: 1.9080 - val_loss: 5.8595 - val_mae: 2.0295
Epoch 4/50
2394/2394 [==============================] - 6s 2ms/step - loss: 4.1575 - mae: 1.7213 - val_loss: 5.8071 - val_mae: 2.0100
Epoch 5/50
2394/2394 [==============================] - 6s 2ms/step - loss: 4.1372 - mae: 1.7070 - val_loss: 5.8084 - val_mae: 2.0090
Epoch 6/50
2394/2394 [==============================] - 6s 3ms/step - loss: 4.1374 - mae: 1.7073 - val_loss: 5.8191 - val_mae: 2.0115
Epoch 7/50
2394/2394 [==============================] - 7s 3ms/step - loss: 4.1369 - mae: 1.7069 - val_loss: 5.8542 - val_mae: 2.0155
Epoch 8/50
2394/2394 [==============================] - 7s 3

In [28]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

895/895 [==============================] - 1s 2ms/step

Values for validation set:
MAE: 2.0102503262034563
MSE: 5.787424790687911
RMSE: 2.4057067133563708
R2:  0.0008294163413137268


Trying big model again

In [9]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
hidden1 = tf.keras.layers.Dense(64, activation='relu')(normalized)
hidden2 = tf.keras.layers.Dense(128, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(64, activation='relu')(hidden2)
output = tf.keras.layers.Dense(2)(hidden3)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 normalization (Normalizatio  (None, 13)               27        
 n)                                                              
                                                                 
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                             

2022-11-20 19:01:50.495813: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-20 19:01:51.313837: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3368 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1


In [10]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])
norm_layer.adapt(data_train_qt)

In [11]:
n_epochs = 25

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/25
2394/2394 [==============================] - 10s 4ms/step - loss: 4.3712 - mae: 1.7388 - val_loss: 6.7053 - val_mae: 2.2076
Epoch 2/25
2394/2394 [==============================] - 8s 3ms/step - loss: 4.0464 - mae: 1.6801 - val_loss: 7.1344 - val_mae: 2.2978
Epoch 3/25
2394/2394 [==============================] - 7s 3ms/step - loss: 4.0061 - mae: 1.6700 - val_loss: 8.3944 - val_mae: 2.4962
Epoch 4/25
2394/2394 [==============================] - 8s 3ms/step - loss: 3.9851 - mae: 1.6642 - val_loss: 8.7805 - val_mae: 2.5423
Epoch 5/25
2394/2394 [==============================] - 7s 3ms/step - loss: 3.9670 - mae: 1.6596 - val_loss: 10.7882 - val_mae: 2.8243
Epoch 6/25
2394/2394 [==============================] - 7s 3ms/step - loss: 3.9514 - mae: 1.6553 - val_loss: 12.7395 - val_mae: 3.0759
Epoch 7/25
2394/2394 [==============================] - 7s 3ms/step - loss: 3.9368 - mae: 1.6525 - val_loss: 11.1440 - val_mae: 2.8839
Epoch 8/25
2394/2394 [==============================] - 8s

In [12]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

895/895 [==============================] - 1s 1ms/step

Values for validation set:
MAE: 1.9538390887126957
MSE: 5.713185347909841
RMSE: 2.3902270494473616
R2:  0.025876856095209677
